# Splink

With inputs from here:
- https://github.com/moj-analytical-services/splink/discussions/2037
- https://github.com/mattjbishop/splink-on-fabric

## Setup

### Install splink

In [1]:
%pip install splink[Spark] -U

StatementMeta(, 9844eabc-2a55-4460-92e5-0fcff684d426, 7, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 79.4 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 135.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 139.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 195.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.5/393.5 kB 109.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



### Imports

In [10]:
#from splink.duckdb.linker import DuckDBLinker
#import splink.duckdb.comparison_library as cl
#import splink.duckdb.comparison_template_library as ctl
#from splink.duckdb.blocking_rule_library import block_on
#from splink.datasets import splink_datasets
#from splink.comparison_level_library import levenshtein_level

StatementMeta(, 2ef33021-87af-479b-94c4-887e7297ccee, 21, Finished, Available, Finished)

Notes from: https://moj-analytical-services.github.io/splink/getting_started.html

In [2]:
import splink.comparison_library as cl
from splink import DuckDBAPI, Linker, SettingsCreator, block_on, splink_datasets

StatementMeta(, 9844eabc-2a55-4460-92e5-0fcff684d426, 9, Finished, Available, Finished)

## Main

### Quickstart

In [3]:
import splink.comparison_library as cl
from splink import DuckDBAPI, Linker, SettingsCreator, block_on, splink_datasets

db_api = DuckDBAPI()

df = splink_datasets.fake_1000

settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        cl.NameComparison("first_name"),
        cl.JaroAtThresholds("surname"),
        cl.DateOfBirthComparison(
            "dob",
            input_is_string=True,
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        cl.EmailComparison("email"),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name", "dob"),
        block_on("surname"),
    ]
)

linker = Linker(df, settings, db_api)

linker.training.estimate_probability_two_random_records_match(
    [block_on("first_name", "surname")],
    recall=0.7,
)

linker.training.estimate_u_using_random_sampling(max_pairs=1e6)

linker.training.estimate_parameters_using_expectation_maximisation(
    block_on("first_name", "surname")
)

linker.training.estimate_parameters_using_expectation_maximisation(block_on("email"))

pairwise_predictions = linker.inference.predict(threshold_match_weight=-5)

clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    pairwise_predictions, 0.95
)

df_clusters = clusters.as_pandas_dataframe(limit=5)

StatementMeta(, 9844eabc-2a55-4460-92e5-0fcff684d426, 10, Finished, Available, Finished)

downloading: https://raw.githubusercontent.com/moj-analytical-services/splink_datasets/master/data/fake_1000.csv
  download progress: 100 %	(==========)
Probability two random records match is estimated to be  0.000821.
This means that amongst all possible pairwise record comparisons, one in 1,218.29 are expected to match.  With 499,500 total possible comparisons, we expect a total of around 410.00 matching pairs
You are using the default value for `max_pairs`, which may be too small and thus lead to inaccurate estimates for your model's u-parameters. Consider increasing to 1e8 or 1e9, which will result in more accurate estimates, but with a longer run time.
----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
  

In [4]:
display(df_clusters)

StatementMeta(, 9844eabc-2a55-4460-92e5-0fcff684d426, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 95d4d231-cecd-49e7-abf7-59a723126b95)